# KoBERT를 이용한 한국어 감정 다중 분류

출처: https://velog.io/@seolini43/KOBERT로-다중-분류-모델-만들기-파이썬Colab



# 기본 설정

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=d5eea5f7cdde9a373082e7d148d404b9ef093d54c45b188815767b445b5c26fa
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-xxtr8ti3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-xxtr8ti3
     |████████████████████████████████| 129 kB 5.0 MB/s 
     |████████████████████████████████| 54.7 MB 30 kB/s 
     |████████████████████████████████| 4.5 MB 49.9 MB/s 
     |████████████████████████████████| 1.2 MB 55.7 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x394f4000 @  0x7fc4757c9615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 881.9 MB 1

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [7]:
#코랩 연결 끊김 방지

'''
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)
'''

'\nfunction ClickConnect(){\n    console.log("코랩 연결 끊김 방지"); \n    document.querySelector("colab-toolbar-button#connect").click() \n}\nsetInterval(ClickConnect, 60 * 1000)\n'

# 데이터 호출 및 전처리

In [8]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [9]:
import pandas as pd
import csv
data = pd.read_csv('/drive/MyDrive/감성대화말뭉치(6개_감정)_Training.csv',encoding="utf-8")

In [10]:
ans = []

#라벨 통일 및 정수화
cnt = 0
for i in range(len(data['Emotions'])):
  cnt += 1
  if data['Emotions'][i] == '기쁨 ':
    data['Emotions'][i] = '기쁨'
  elif data['Emotions'][i] == '불안 ':
    data['Emotions'][i] = '불안'

for i in data['Emotions']:
  if i not in ans:
    ans.append(i)

print(ans)
print(cnt)
print(len(ans))

['기쁨', '불안', '당황', '슬픔', '분노', '상처']
40879
6


In [11]:
data['Emotions'] = data['Emotions'].replace(['기쁨', '불안', '당황', '슬픔', '분노', '상처'],[0,1,2,3,4,5])

In [15]:
#데이터 랜덤으로 섞기
data=data.sample(frac=1).reset_index(drop=True)

In [16]:
data.head()

,Emotions,Sentences
0,3,아이들은 아직 한참 어린데 회사 생활은 위태롭고 하니 많이 우울해져.
1,3,퇴직을 권유받았어.
2,5,젊은이들을 보면 늙고 힘없는 내가 미워져.
3,0,펀드매니저인 내 친구 아들에게 내 보험금을 맡겨볼까 해.
4,5,집에 오면 나는 투명인간 같아.


In [17]:
data_list = []
for q, label in zip(data['Sentences'], data['Emotions'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [18]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['아이들은 아직 한참 어린데 회사 생활은 위태롭고 하니 많이 우울해져.', '3']
['친구한테 배신당한 기분이야.', '5']
['시험을 잘 보기 위해서 잠도 안 자고 노력했는데 성적이 그만큼 나오지 않아서 괴로워.', '5']
['오늘 병원을 갔다 왔어.', '1']
['나 때문에 친구가 넘어져서 뼈가 부러졌어. 병원에 입원해있는데 무슨 말을 해야 할지 모르겠어.', '5']
['이 동네 노인들은 정말 답답해.', '4']
['내가 결혼을 생각한 사람에게 오늘 이별 통보를 받아서 너무 충격이야.', '5']


# Train & Test DATA

In [20]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0)
print(len(dataset_train))
print(len(dataset_test))

36791
4088


In [43]:
df = pd.DataFrame(dataset_test)
df.to_csv('6emotions_test.csv')

# KoBERT 입력데이터

In [21]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [22]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [23]:
#입력 데이터 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [24]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습모델 생성

In [27]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   #클래스 수
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [28]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 모델 학습 및 저장

In [29]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/575 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8764739036560059 train acc 0.171875
epoch 1 batch id 201 loss 1.392524242401123 train acc 0.2737873134328358
epoch 1 batch id 401 loss 1.2215123176574707 train acc 0.4183291770573566
epoch 1 train acc 0.4758216403162055


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/64 [00:00<?, ?it/s]

...

epoch 9 test acc 0.6463099888392857


  0%|          | 0/575 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.14282535016536713 train acc 0.953125
epoch 10 batch id 201 loss 0.3395363390445709 train acc 0.9319807213930348
epoch 10 batch id 401 loss 0.31988653540611267 train acc 0.9342269326683291
epoch 10 train acc 0.9377900197628459


  0%|          | 0/64 [00:00<?, ?it/s]

epoch 10 test acc 0.6467633928571429


In [30]:
#model.save_pretrained('/drive/MyDrive/sentiment_calssfier_ko') #모델저장
#tokenizer.save_pretrained('/drive/MyDrive/sentiment_calssfier_ko') #토크나이저 저장

import os
os.chdir('/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)')
os.getcwd()

path = '/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)'
torch.save(model, '/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)/6emotions_model.pt')  # 전체 모델 저장

torch.save(model.state_dict(), '/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)/6emotions_model_state_dict.pt')  # 모델 객체의 state_dict 저장

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)/6emotions_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

# 모델 호출 및 테스트

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

# torch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기 필수
bertmodel, vocab = get_pytorch_kobert_model()


# KoBERT에 입력될 데이터셋 정리
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

# 모델 정의
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [33]:
#★★★현재경로가 model이 있는 폴더여야함★★★
import os
os.chdir('/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)')

model1 = torch.load('/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)/6emotions_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)/6emotions_model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('/drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)/6emotions_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /drive/MyDrive/sentiment_calssfier_ko(epoch10,batch64)/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
#'기쁨', '불안', '당황', '슬픔', '분노', '상처'

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 1:
                test_eval.append("불안")
            elif np.argmax(logits) == 2:
                test_eval.append("당황")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 4:
                test_eval.append("분노")
            elif np.argmax(logits) == 5:
                test_eval.append("상처")

        #print(">> 입력하신 내용에서 " + test_eval[0] + "이 느껴집니다.")
        ans.append(test_eval[0])


          

In [ ]:

import pandas as pd
import csv
data = pd.read_csv('/content/6emotions_test.csv',encoding="utf-8")

ans = []

for i in data['Sentences']:
  predict(i)


for i in range(len(ans)):
  if ans[i] == data['Emotions'][i]:
    cnt += 1

print(cnt)
  

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 나 내일 군대 가
>> 입력하신 내용에서 슬픔 느껴집니다.


하고싶은 말을 입력해주세요 : 고기 먹자
>> 입력하신 내용에서 기쁨 느껴집니다.


하고싶은 말을 입력해주세요 : 나 집에 갈래
>> 입력하신 내용에서 상처 느껴집니다.


하고싶은 말을 입력해주세요 : 수업 너무 재미없다
>> 입력하신 내용에서 슬픔 느껴집니다.


하고싶은 말을 입력해주세요 : 치킨 도착하면 불러줘
>> 입력하신 내용에서 분노 느껴집니다.


하고싶은 말을 입력해주세요 : 나랑 장난해?
>> 입력하신 내용에서 분노 느껴집니다.


하고싶은 말을 입력해주세요 : 나 발가락 뿌러짐ㅋㅋ
>> 입력하신 내용에서 분노 느껴집니다.


하고싶은 말을 입력해주세요 : 발가락 부러짐
>> 입력하신 내용에서 상처 느껴집니다.


하고싶은 말을 입력해주세요 : 발가락 부러짐ㅋㅋ
>> 입력하신 내용에서 불안 느껴집니다.


하고싶은 말을 입력해주세요 : 어이없네
>> 입력하신 내용에서 분노 느껴집니다.


하고싶은 말을 입력해주세요 : 너 지금 뭐하는거야
>> 입력하신 내용에서 슬픔 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘 회사에서 개빡침
>> 입력하신 내용에서 불안 느껴집니다.


하고싶은 말을 입력해주세요 : 오늘 회사에서 혼났어
>> 입력하신 내용에서 상처 느껴집니다.


하고싶은 말을 입력해주세요 : 나 상받았어
>> 입력하신 내용에서 기쁨 느껴집니다.


하고싶은 말을 입력해주세요 : 나 1등 했어
>> 입력하신 내용에서 기쁨 느껴집니다.


하고싶은 말을 입력해주세요 : 나 내일 전역이야
>> 입력하신 내용에서 슬픔 느껴집니다.




KeyboardInterrupt: ignored